In [6]:
import matplotlib.pyplot as plt
import pandas as pd 

In [45]:
data = pd.read_excel('data.xlsx')

In [48]:
data

,number,comment,likes,sentiment
0,2,0:13 0:13 내 수술 들어올래? 이거 진짜 미친 거 아님? 누가 수술 들어올래...,59,1
1,2,조정석 너무잘생김 ..,32,1
2,2,화신이가 수술들어오라는 것 같앜ㅋㅋㅋㅋ♡♡ 짜릿햌ㅋㅋㅋㅌㅋ,13,1
3,2,내 심장부터 먼저 수술해라...,13,1
4,2,미처ㅠㅠㅠㅠㅠㅠㅠㅠ이 영상보고 오나귀 다시 정주행하러 달려갑니다ㅜㅜㅜㅜㅜ하ㅜㅜ,8,1
...,...,...,...,...
771,12,English pleyazzzzz,0,0
772,12,아..... 송화 치홍에 미련을 못버렸었는데.... 오늘 폭망이네.... 송화 익준...,1,0
773,12,익송 익송 익송 익송 익송 익송 익송 익송 익송 익송 익송 익송 익송 익송 익송 익...,0,0
774,12,솔직히 채송화 반말하는 안치홍한테 설렜잖아ㅜㅜ 설렜다고 말해!! 왜 말을 못하냐고!!,3,0


In [69]:
data['sentiment'] = 0
data.loc[data['likes']>10,'sentiment']=1

In [70]:
word_dict = {}

number = 0 
for d in data['comment']:
    d = str(d)
    sent = d.split(' ')
    for w in sent:
        if w in word_dict:
            word_dict[w][1] +=1
        else:
            word_dict[w] = [number, 1]
            number+=1

In [71]:
# Number of Sentences S
# Number of Classes N

# Assumes there are more than 100 sentencces
S = len(data)
low  = 0.05*S
high = 1.0*S

word_dict2 = {}
number = 0
for w in word_dict:
  if low < word_dict[w][1] < high:
    word_dict2[w] = word_dict[w]
    word_dict2[w][0] = number
    number+=1


print(len(word_dict))
print(len(word_dict2))

5306
18


In [72]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def category(v):
  if v =='positive': 
    return 1
  else:
    return 0

#y = np.array(list(map(category, data['sentiment'])))
y= data['sentiment'].values
x = np.zeros((len(data), len(word_dict)))

# Make X
for i, d in enumerate(data['comment']):
  d= str(d)
  sent = d.split(' ')
  for w in sent:
      try:
        index  = word_dict[w][0]
        x[i, index] +=1   
      except:
        pass

# Normalize
scaler = MinMaxScaler()
scaler.fit(x)
x = scaler.transform(x)
print(f"X:{x.shape}")
print(f"y:{y.shape}")

X:(776, 5306)
y:(776,)


In [73]:
# Models 
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=.3,random_state=0)


svc = LinearSVC(random_state=0, tol=1e-5, verbose=1)
gaunb = GaussianNB()
dt = tree.DecisionTreeClassifier()
rf = RandomForestClassifier()

# Training 


svc.fit(X_train, y_train).decision_function(X_test)
print(f"SVC Accuracy: {accuracy_score(svc.predict(X_test),y_test)}")
rf.fit(X_train, y_train)
print(f"RF Accuracy: {accuracy_score(rf.predict(X_test),y_test)}")

[LibLinear]SVC Accuracy: 0.703862660944206
RF Accuracy: 0.7510729613733905


In [74]:
temp = data['likes']>20
temp.sum()

172

In [88]:
lst = []
for i,j in enumerate(rf.feature_importances_):
    if j>0.006:
        lst.append(i)
lst
    

[205, 275, 359, 1377, 2904, 3837]

In [89]:
for j,i in word_dict.items():
    if i[0] in lst:
        print(j)

ㅠㅠㅠㅠ
오빠랑
사람
preview
곰곰커플!!!!
99즈오래가자영원하자사랑해코로나잠잠해지면티비엔팬미팅해야하는거날즤?이대로가지마라


In [75]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

print(classification_report(y_test, rf.predict(X_test)))
confusion_matrix(y_test, rf.predict(X_test))

precision    recall  f1-score   support

           0       0.76      0.97      0.85       172
           1       0.62      0.13      0.22        61

    accuracy                           0.75       233
   macro avg       0.69      0.55      0.53       233
weighted avg       0.72      0.75      0.69       233



array([[167,   5],
       [ 53,   8]], dtype=int64)